## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
from pprint import pprint
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kholodnyy,RU,62.72,147.95,12.58,94,100,2.82,light snow
1,1,Chuy,UY,-33.70,-53.46,52.90,82,100,5.93,overcast clouds
2,2,Pisco,PE,-13.70,-76.22,62.01,87,75,13.87,broken clouds
3,3,Hilo,US,19.73,-155.09,77.00,88,90,1.79,overcast clouds
4,4,Grand Gaube,MU,-20.01,57.66,73.99,52,0,15.01,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Enter minimum temperature:"))
max_temp = float(input("Enter maximum temperature:"))

Enter minimum temperature:75
Enter maximum temperature:91


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] > min_temp)]
clean_df

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Hilo,US,19.73,-155.09,77.00,88,90,1.79,overcast clouds
6,6,Yulara,AU,-25.24,130.99,84.20,7,37,12.75,scattered clouds
7,7,Carnarvon,AU,-24.87,113.63,78.80,65,0,19.46,clear sky
19,19,Bossangoa,CF,6.49,17.46,76.50,82,61,4.07,broken clouds
23,23,Bathsheba,BB,13.22,-59.52,80.60,83,75,20.80,shower rain
...,...,...,...,...,...,...,...,...,...,...
685,685,Bam,IR,29.11,58.36,76.19,16,0,7.76,clear sky
688,688,Padang,ID,-0.95,100.35,85.17,65,94,4.47,overcast clouds
690,690,Kingston,JM,18.00,-76.79,80.60,78,20,8.05,thunderstorm
691,691,Amahai,ID,-3.33,128.92,84.11,68,80,5.75,light rain


In [5]:
# 4a. Determine if there are any empty rows.
clean_df.isnull().sum()

City ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
new_df = clean_df.dropna()
new_df.count()

City ID                214
City                   214
Country                214
Lat                    214
Lng                    214
Max Temp               214
Humidity               214
Cloudiness             214
Wind Speed             214
Current Description    214
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Hilo,US,77.00,overcast clouds,19.73,-155.09,
6,Yulara,AU,84.20,scattered clouds,-25.24,130.99,
7,Carnarvon,AU,78.80,clear sky,-24.87,113.63,
19,Bossangoa,CF,76.50,broken clouds,6.49,17.46,
23,Bathsheba,BB,80.60,shower rain,13.22,-59.52,
27,Birao,CF,82.67,clear sky,10.28,22.79,
41,Kavieng,PG,81.14,overcast clouds,-2.57,150.80,
44,Brejo Santo,BR,78.80,clear sky,-7.49,-38.99,
45,Kavaratti,IN,83.46,clear sky,10.57,72.64,
46,Labuhan,ID,84.07,moderate rain,-6.88,112.21,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotel_data = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_name = hotel_data['results'][0]['name']
        hotel_df.loc[index,'Hotel Name'] = hotel_name
    except IndexError:
        print('Hotel name not found for city: ',row['City'],'-Skipping')
        pass

Hotel name not found for city:  Birao -Skipping
Hotel name not found for city:  Goundam -Skipping
Hotel name not found for city:  Ati -Skipping
Hotel name not found for city:  Mach -Skipping
Hotel name not found for city:  Tessalit -Skipping
Hotel name not found for city:  Tenenkou -Skipping
Hotel name not found for city:  Damphu -Skipping
Hotel name not found for city:  Raga -Skipping
Hotel name not found for city:  Bousso -Skipping
Hotel name not found for city:  Adrar -Skipping
Hotel name not found for city:  Salme -Skipping
Hotel name not found for city:  Thinadhoo -Skipping
Hotel name not found for city:  Matongo -Skipping
Hotel name not found for city:  Merauke -Skipping
Hotel name not found for city:  Filingue -Skipping


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Hilo,US,77.0,overcast clouds,19.73,-155.09,Hilo Hawaiian Hotel
6,Yulara,AU,84.2,scattered clouds,-25.24,130.99,Desert Gardens Hotel - Ayers Rock Resort
7,Carnarvon,AU,78.8,clear sky,-24.87,113.63,Hospitality Carnarvon
19,Bossangoa,CF,76.5,broken clouds,6.49,17.46,LA PERLE DE BOSTON
23,Bathsheba,BB,80.6,shower rain,13.22,-59.52,Atlantis Hotel


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))